In [72]:
# !unzip '/content/drive/My Drive/Colab Notebooks/cotton_disease/data.zip' -d '/content/drive/My Drive/Colab Notebooks/cotton_disease/'

In [73]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [74]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [75]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob

In [76]:
dataset_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/'
training_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/train/'
validation_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/val/'
test_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/test/'
model_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/inception_model.h5'

IMAGE_SIZE = [224, 224,3]
EPOCHS = 30
TARGET_SIZE = (224,224)
BATCH_SIZE = 16

In [77]:
# useful for getting number of output classes
folders = glob(training_path+'*')
print(len(folders))

4


In [78]:
# importing the Inception V3 and using the imagenet weights
# Here we will be using imagenet weights

inception = InceptionV3(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE)

In [79]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [80]:
# our layers - you can add more if you want
x_network = Flatten()(inception.output)
# adding the prediction layer
prediction_layer = Dense(len(folders),activation='softmax')(x_network)

In [81]:
# create a model object
model = Model(inputs = inception.input , outputs = prediction_layer)
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 111, 111, 32) 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 111, 111, 32) 0           batch_normalization_282[0][0]    
_______________________________________________________________________________________

In [82]:
# compiling the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [83]:
# creating callbacks to save best model
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath= model_path,
                                                               save_weights_only='False',
                                                               monitor='val_accuracy',
                                                               save_best_only=True)


In [84]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs = None):
    k = list(logs.keys())
    # print(k)
    if(logs.get('val_accuracy') > 0.99):
      print('Required accuracy reached , so stopping training.....!!')
      self.model.stop_training = True
  

In [85]:
# creating datagenerators

# train datagenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# test datagenerator
test_datagen = ImageDataGenerator(rescale= 1./255)

In [86]:
training_data = train_datagen.flow_from_directory(training_path,
                                                  target_size = TARGET_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode = 'categorical')

validation_dataset = test_datagen.flow_from_directory(validation_path,
                                                   target_size = TARGET_SIZE,
                                                   batch_size = BATCH_SIZE,
                                                   class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.


In [87]:
# fit the model

r = model.fit_generator(training_data,
                        validation_data = validation_dataset,
                        epochs = EPOCHS,
                        steps_per_epoch = len(training_data),
                        validation_steps = len(validation_dataset),
                        callbacks=[model_checkpoint_callback , MyCallback()])

Epoch 1/30
122/122 [==============================] - 41s 340ms/step - loss: 2.0933 - accuracy: 0.7678 - val_loss: 1.2135 - val_accuracy: 0.8580
Epoch 2/30
122/122 [==============================] - 43s 353ms/step - loss: 1.1439 - accuracy: 0.8662 - val_loss: 0.8574 - val_accuracy: 0.9105
Epoch 3/30
122/122 [==============================] - 43s 351ms/step - loss: 0.8566 - accuracy: 0.9016 - val_loss: 0.6749 - val_accuracy: 0.9198
Epoch 4/30
122/122 [==============================] - 43s 350ms/step - loss: 1.1275 - accuracy: 0.9001 - val_loss: 0.5955 - val_accuracy: 0.9321
Epoch 5/30
122/122 [==============================] - 40s 326ms/step - loss: 0.5877 - accuracy: 0.9344 - val_loss: 0.7000 - val_accuracy: 0.9228
Epoch 6/30
122/122 [==============================] - 39s 323ms/step - loss: 0.9938 - accuracy: 0.9221 - val_loss: 1.8759 - val_accuracy: 0.8735
Epoch 7/30
122/122 [==============================] - 40s 324ms/step - loss: 1.2046 - accuracy: 0.9103 - val_loss: 1.4884 - val_ac